In [7]:
import os, re, csv
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi

In [9]:
# 데이터셋 불러와서 정리하기

eta_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\질문.csv", encoding='ANSI')
eta_df = eta_df.drop(['Unnamed: 0', '글쓴이', '더보기', 'id', '작성시간', '좋아요', '댓글 수'], axis=1)
eta_df['TAGSET'] = np.nan
eta_df.rename(columns={"내용":"QUESTION", "댓글":"ANSWER"}, inplace=True)

eta2_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\질문2.csv", encoding='ANSI')
eta2_df = eta2_df.drop(['Unnamed: 0', '댓글쓴이', '대댓글여부', 'id'], axis=1)
eta2_df['TAGSET'] = np.nan
eta2_df.rename(columns={"내용":"QUESTION", "댓글":"ANSWER"}, inplace=True)

dm_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\이중전과게시판_이중검색.csv", encoding='UTF-8')
dm_df = dm_df.drop(['Unnamed: 0', '댓글쓴이', '대댓글여부', 'id'], axis=1)
dm_df['TAGSET'] = np.nan
dm_df.rename(columns={"내용":"QUESTION", "댓글":"ANSWER"}, inplace=True)

# 데이터 병합
df_lst = [eta_df, eta2_df, dm_df]
merged_df = pd.concat(df_lst)
merged_df.reset_index(drop=True, inplace=True)
merged_df = merged_df[['QUESTION', "ANSWER", "TAGSET"]]

In [10]:
question_list = merged_df['QUESTION'].tolist()

In [4]:
## pycrfsuite 띄어쓰기 교정
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer

to_feature = CharacterFeatureTransformer(
    TemplateGenerator(
        begin=-2,
        end=2,
        min_range_length=3,
        max_range_length=3)
    )

In [4]:
## 훈련 파일 정제 (수강편람 + 토지)
train_path = r'E:\Programming\python\NLP\howls_nlp\Chatbot\자료\space_trainer.txt'

corpus = open(train_path, encoding='utf-8').read()
corpus = corpus.replace('\t',' ')
train_corpus = corpus.split('\n')
print(train_corpus[:30])

['14장 추적', '귀녀의 모습을 한번 쳐다보고 떠나려 했다. 집안을 이리저리 기웃거리던 강포수는 윤씨부인에게 인사를 올리고 중문을 나서는 치수 뒷모습을 보았다. 실망에 얼굴이 일그러지면서 강포수는 치수 뒤를 따라나올 수밖에 없었다. 문전에는 하인들이 즐비하게 늘어서서 상전을 전송하기 위해 기다리고 있었다.', '치수는 거들떠보지 않고 나귀 등에 오르고 말고삐를 잡은 수동은 엉성한 수염에 묻힌 작은 입술을 다물고 늘어선 하인들에게 일별을 던진다.', '삼수가 묘하게 웃었다. 복이 삼수에게 곁눈질을 했다. 짐을 실은 다른 한 필의 나귀는 텁석부리 강포수를 경계했던지 말고삐를 잡자 코를 불었다.', '"나리마님, 안녕히 다녀오시오."', '제가끔 인사했으나 최치수는 말안장에 앉아 강물만 바라보고 있었다.', '의외로 단촐한 출발이었다. 하인들이 서너 명은 수행하리니, 김평산도 동행하지 않을까 예상했었지만 기운 센 편도 아닌 수동이 한 사람만 데리고 강포수와 함께 출발한 것이다. 엉거주춤 서 있는 하인들은 언덕길을 내려가는 두 필의 나귀를 멍청히 내려다본다.', '일행은 강을 끼고, 나귀와 사람들 발자국에 다져진 길을 천천히 간다. 몇 번씩이나 돌아보고 하며 아쉬움에 가득 찼던 강포수는 최참판댁 지붕이 시야에서 사라지자 눈을 내리깔았다. 고개가 차츰 숙어지면서 발끝으로 눈길이 떨어진다.', "'머 영영 안 돌아올 기건데?'", '나귀는 강포수에게 곁눈질을 하며 끝내 미덥지 못한가 코를 분다. 앞서가는 나귀 등에는 최치수의 뒷모습이 짐짝같이 보이었다.', "'온 천지가 허퉁한 거맨치로 마음이 허퉁해서 갈 바를 못 잡겄다. 총도 싫고 돈도 싫고 ……'", '길섶에 뻗은 풀잎에 이슬이 남아 있어서 짚세기가 젖는다. 날씨는 변덕을 부릴는지, 흩어져 있는 구름은 훨씬 빠르게 움직이고 있는 것 같았으나 하늘은 휑하니 높았고 푸른 빛은 차가웠다. 들판은 바람에 일렁이고 있었다. 목동이 소를 몰고 들판을 질러가는데 송아지가 어미소 뒤를 졸래졸래 따라간다.', '십리 길을 거의 

In [8]:
# 모델 훈련
from pycrfsuite_spacing import PyCRFSuiteSpacing

model_path = 'space_model.crfsuite'
correct = PyCRFSuiteSpacing(
    to_feature=to_feature,
    l1_cost = 0,
    l2_cost = 1.0,  # L2 규제 사용
    verbose=False,
)
correct.train(train_corpus, model_path)

In [7]:
correct('나도너만큼잘할수있어멍청아!')

'나도너만큼잘할 수 있어멍청아!'

In [24]:
corrected_question = []
for x in question_list:
    correction = correct(x)
    corrected_question.append(correction)
print(corrected_question[:10])

['조교님! 이중전공 이랑 부전공 차이에 대해서 궁금해요!이중전공은 학위+ 졸업장 표시인건 알겠는 데부전공은 졸업장에 도 표시가 안되나요?', '안녕하세요 조교님!재수강 연결 방밥 대로 종합정보시스템- 수업수강관리- 재수강과목선택메뉴를 들어갔는 데안내사항 및 이전 성적 연결 현황에 ‘동일 교과로 자동연결 되었습다 ‘ 라고 나와 있습니다. 그럼 학사공지에 나와 있는 대로 재수강할 이전 성적을 선택해 재수강연결을 할 필요없이 아무것도 안해도 된 다는 건가요? 감사합니다. !', '조교님 안녕하세요!! 항상 수고가 많으십니다 다름아니라 제가 총 평점이 좀 낮은 편이라 막학기에 수업을 더 들으면서 평점을 올려보고자 하는 데요, 졸업학점인 134학점을 초과해서 듣는 것이 문제가 되나요??ㅠㅠ', '안녕하세요 조교님!제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 변경하려고 시도했는 데자체시험에 2번 이나 떨어져 변경을 실패했습니다그래서 지난 겨울 방학(2019년)에 부전공으로 경영을 선택을 하고 2020년도 1학기 이중전공 변경기간에 경영으로 신청하려고 합니다한 번 종합정보시스템을 들어갔는 데 사진과 같은 내용이 있더라고요신경쓰지 않고 변경기간에 경영으로 신청하면 되는 건가요?', '이중전공을 바꾸고싶은 데 혹시 바꾸게 된 다면 그전에 받은 학점은 이어가는 건가요? 아니면 소멸되는 건가요?', '조교님 혹시 등록휴학 후 자퇴할 경우 이번 학기 등록금 환불이 가능한 가요?', '졸업유예 기간 동안 학점 취득이 가능한 가요?', '조교님 제가 인간과 사회 2학점 짜리 과목 씨쁠을 맞았었는 데 이걸 언어와 문학 3학점 짜리 페논패 과목으로 재수강 연결이 가능하나요??', '안녕하세요 조교님. 2019-2학기에 8학기 까지 마치고, 현재 사정상 2학점이 남은 상태로 휴학 중 입니다. 전공과 이중전공 학점은 다 채운 상황이나, 총 취득학점이 132학점 이라 2학점이 모자랍니다. 추가 학기를 들어야 하는 상황인데요, 이 경우 2020 여름 계절학기 때 2학점 수강

In [12]:
## soyspace 이용한 띄어쓰기
from soyspacing.countbase import CountSpace

corpus_fname = r'D:\Shin\Develop\python\NLP\Resource\big_space_trainer.txt'
model = CountSpace()
model.train(corpus_fname)

# 모델 저장하기
model.save_model(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\models\soy_big_space_model", json_format=False)

all tags length = 154633 --> 124077, (num_doc = 3460682)

In [13]:
# 모델 불러오기
from soyspacing.countbase import CountSpace

model = CountSpace()
model.load_model(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\models\soy_big_space_model", json_format=False)

In [14]:
soy_corrected_question = []
for x in question_list:
    corrected = model.correct(x)
    soy_corrected_question.append(corrected)
print(soy_corrected_question[:10])

[('조교님! 이중전공이랑 부전공 차이에 대해서 궁금해요!이중전공은 학위+졸업장 표시인건 알겠는데부전공은 졸업장에도 표시가 안되나요?', [None, None, None, 1, None, 0, 0, 0, None, 1, None, 0, 1, None, None, 1, 0, 0, 1, None, 0, None, None, None, None, 0, 0, 0, 1, None, None, None, None, None, 1, None, None, None, 1, None, None, 0, None, None, 0, 0, 1, None, None, None, None, 1, None, None, 1, None, None, None, 0, 1]), ('안녕하세요 조교님!재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴를 들어갔는데안내사항 및 이전성적 연결현황에 ‘동일교과로 자동연결되었습다 ‘ 라고 나와 있습니다. 그럼 학사공지에 나와 있는 대로 재수강할 이전 성적을 선택해 재수강연결을 할 필요 없이 아무것도 안 해도 된다는 건가요? 감사합니다.!', [0, 0, 0, None, 1, None, None, None, None, None, 0, 1, None, None, None, 1, None, 1, None, 0, 0, 0, 0, 0, None, None, None, None, None, None, None, None, None, None, 0, None, 0, None, None, None, None, None, 1, 0, 0, 0, 0, None, None, None, None, 1, 1, None, None, None, 1, None, None, None, None, 1, None, None, None, None, None, 1, None, None, None, None, None, None, None, 1, 1, None, 1, None, 1, 0, 0, 0, 0, 1, 0, 1, None, 0, 0, None, 1, 0, 1, 0, 1, 0, 

In [7]:
spaced_quest = []
for quest, tag in soy_corrected_question:
    if type(quest) == str:
        spaced_quest.append(quest)
    else:
        continue
print(spaced_quest[:10])

['조교님! 이중전공이랑 부전공 차이에 대해서 궁금해요!이중전공은 학위+졸업장 표시인건 알겠는데부전공은 졸업장에도 표시가 안되나요?', '안녕하세요 조교님!재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴를 들어갔는데안내사항 및 이전성적 연결현황에 ‘동일교과로 자동연결되었습다 ‘ 라고 나와 있습니다. 그럼 학사공지에 나와 있는 대로 재수강할 이전 성적을 선택해 재수강연결을 할 필요 없이 아무것도 안해도 된다는 건가요? 감사합니다.!', '조교님 안녕하세요!! 항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라 막학기에 수업을 더 들으면서 평점을 올려보고자 하는데요, 졸업학점인 134학점을 초과해서 듣는것이 문제가 되나요??ㅠㅠ', '안녕하세요 조교님!제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 변경하려고 시도했는데자체시험에 2번이나 떨어져 변경을 실패했습니다그래서 지난 겨울방학(2019년)에 부전공으로 경영을 선택을 하고 2020년도 1학기 이중전공 변경기간에 경영으로 신청하려고 합니다한번 종합정보시스템을 들어갔는데 사진과 같은 내용이 있더라고요신경 쓰지 않고 변경기간에 경영으로 신청하면 되는 건가요?', '이중전공을 바꾸고 싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요? 아니면 소멸되는 건가요?', '조교님 혹시 등록휴학 후 자퇴할 경우 이번학기 등록금 환불이 가능한가요?', '졸업유예 기간동안 학점 취득이 가능한가요?', '조교님 제가 인간과 사회 2학점 짜리 과목 씨쁠을 맞았었는데 이걸 언어와 문학 3학점 짜리 페논패 과목으로 재수강 연결이 가능하나요??', '안녕하세요 조교님.2019-2학기에 8학기 까지 마치고, 현재 사정상 2학점이 남은 상태로 휴학중입니다. 전공과 이중전공 학점은 다 채운 상황이나, 총 취득학점이 132학점이라 2학점이 모자랍니다. 추가학기를 들어야 하는 상황인데요, 이 경우 2020 여름 계절학기 때 2학점 수강 후 수료가 가능한지 궁금합니다. 답변 부탁드립니다-!',

In [8]:
merged_df['QUESTION'] = spaced_quest

merged_df

,QUESTION,ANSWER,TAGSET
0,조교님! 이중전공이랑 부전공 차이에 대해서 궁금해요!이중전공은 학위+졸업장 표시인건...,NaN,NaN
1,안녕하세요 조교님!재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴...,NaN,NaN
2,조교님 안녕하세요!! 항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라...,NaN,NaN
3,안녕하세요 조교님!제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 ...,NaN,NaN
4,이중전공을 바꾸고 싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요? ...,NaN,NaN
...,...,...,...
6885,안녕하세요 중외통 17학번인데요이중전공 어떤 걸로 하면 좋은지 추천해주세요 ㅎㅎ,['아 그렇군요! 감사합니다!!'],NaN
6886,공대생 이중전공으로 언어할까 하는데 힘듦? 언어추천좀,['스페인어 했다가 힘들어서 영어로 바꾼 사람은 봤음'],NaN
6887,국제통상학과 이중전공생인데 걱정이 많이 된다..,"['저도요.... 진짜 국통법 때문에 미칠거 같네요 ,', '나두...', '국통 ...",NaN
6888,경영 후기 이중전공 신청할 때이중전공 떨어져서 부전공 경영인데후기 이중 경영 신청해...,"['부전공학점 다 채운다음 이중전공이랑 차이나는 전공점수 채워야되요', '한마디로 ...",NaN


In [9]:
### 정규식(이모티콘)으로 전처리하기 ###

In [10]:
norm_content = []
for text in spaced_quest:
    punct = re.compile(r'[\~\'\"\.\,\?\!\:\;\)\(\+]+') 
    norm_text = re.sub(punct, "", text)
    norm_text = re.sub(r'(안녕)+(하세요)?', '', norm_text)
    norm_text = re.sub(r'(조교)[님]?', "", norm_text)
    norm_text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', norm_text) # ㅠ하나 쓰는 놈들꺼 제거...
    norm_text = re.sub( r"\s{2,}"," ", norm_text) 
    norm_text = re.sub( r"\A\s+",'', norm_text)
    norm_text = re.sub(r"([가-힣]+)[여용염]",r"\1요",norm_text) # 용용체 제거 슈바끄...
    norm_text = re.sub(r"\b이중[^전공]*?\b","이중전공",norm_text) # 이중으로만 적혀있는거 변형
    norm_text = re.sub(r"\b부전[^공]*?\b","부전공",norm_text) # 부전으로만 적혀있는거 변형

    # 줄인말제거
    norm_text = re.sub(r"\b전심[가-힣]*\b", "전공심화", norm_text)
    norm_text = re.sub(r"\b융전[가-힣]*\b", "융합전공", norm_text)
    norm_text = re.sub(r'\b연계전공\b', '융합전공', norm_text) # 단어 통일화
    
    norm_content.append(norm_text)

print(norm_content[:5])

merged_df['QUESTION'] = norm_content

merged_df

['이중전공이랑 부전공 차이에 대해서 궁금해요이중전공은 학위졸업장 표시인건 알겠는데부전공은 졸업장에도 표시가 안되나요', '재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴를 들어갔는데안내사항 및 이전성적 연결현황에 ‘동일교과로 자동연결되었습다 ‘ 라고 나와 있습니다 그럼 학사공지에 나와 있는 대로 재수강할 이전 성적을 선택해 재수강연결을 할 필요 없이 아무것도 안해도 된다는 건가요 감사합니다', '항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라 막학기에 수업을 더 들으면서 평점을 올려보고자 하는데요 졸업학점인 134학점을 초과해서 듣는것이 문제가 되나요', '제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 변경하려고 시도했는데자체시험에 2번이나 떨어져 변경을 실패했습니다그래서 지난 겨울방학2019년에 부전공으로 경영을 선택을 하고 2020년도 1학기 이중전공 변경기간에 경영으로 신청하려고 합니다한번 종합정보시스템을 들어갔는데 사진과 같은 내요이 있더라고요신경 쓰지 않고 변경기간에 경영으로 신청하면 되는 건가요', '이중전공을 바꾸고 싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요 아니면 소멸되는 건가요']


,QUESTION,ANSWER,TAGSET
0,이중전공이랑 부전공 차이에 대해서 궁금해요이중전공은 학위졸업장 표시인건 알겠는데부전...,NaN,NaN
1,재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴를 들어갔는데안내사...,NaN,NaN
2,항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라 막학기에 수업을 더 ...,NaN,NaN
3,제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 변경하려고 시도했는...,NaN,NaN
4,이중전공을 바꾸고 싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요 아...,NaN,NaN
...,...,...,...
6885,중외통 17학번인데요이중전공 어떤 걸로 하면 좋은지 추천해주세요,['아 그렇군요! 감사합니다!!'],NaN
6886,공대생 이중전공으로 언어할까 하는데 힘듦 언어추천좀,['스페인어 했다가 힘들어서 영어로 바꾼 사람은 봤음'],NaN
6887,국제통상학과 이중전공생인데 걱정이 많이 된다,"['저도요.... 진짜 국통법 때문에 미칠거 같네요 ,', '나두...', '국통 ...",NaN
6888,경영 후기 이중전공 신청할 때이중전공 떨어져서 부전공 경영인데후기 이중전공 경영 신...,"['부전공학점 다 채운다음 이중전공이랑 차이나는 전공점수 채워야되요', '한마디로 ...",NaN


In [11]:
### kiwi를 사용한 형태소 분석 ###

In [17]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

#대분류 사전 추가
kiwi.add_user_word('이중전공', 'DM', 20.0)
kiwi.add_user_word('융합전공', 'UM', 20.0)
kiwi.add_user_word('전공심화', 'MD', 20.0)
kiwi.add_user_word('부전공', 'SM', 20.0)
kiwi.add_user_word('연계전공', 'UM', 20.0)

kiwi.prepare()

## 형태소 분석
sent = ''
lem_cont = []
for quest in norm_content:
    ## 형태소만 떼어내기
    temp_tagging = [x[0] for x in kiwi.analyze(quest, top_n=1)]
    inner_temp = ["{}/{}".format(word, tag) for word, tag, score1, score2 in temp_tagging[0]]
    lem_cont.append(inner_temp)

print(lem_cont[:32])

[['이중전공/@', '이랑/JC', '부전공/@', '차이/NNG', '에/JKB', '대하/VV', '어서/EC', '궁금/XR', '하/XSA', '어요/EF', '이중전공/@', '은/JX', '학위/NNG', '졸업장/NNG', '표시/NNG', '이/VCP', 'ᆫ/ETM', '거/NNB', 'ᆫ/JX', '알/VV', '겠/EP', '는데/EC', '부전공/@', '은/JX', '졸업장/NNG', '에/JKB', '도/JX', '표시/NNG', '가/JKS', '안/MAG', '되/VV', '나요/EF'], ['재수강/NNP', '연결/NNG', '방/NNG', '밥/NNG', '대로/JX', '종합/NNG', '정보/NNG', '시스템/NNG', '-/SO', '수업/NNG', '수강/NNG', '관리/NNG', '-/SO', '재/XPN', '수강/NNG', '과목/NNG', '선택/NNG', '메뉴/NNG', '를/JKO', '들어가/VV', '었/EP', '는데/EC', '안내/NNG', '사항/NNG', '및/MAG', '이전/NNG', '성적/NNG', '연결/NNG', '현황/NNG', '에/JKB', '‘/SS', '동일/NNG', '교과/NNG', '로/JKB', '자동연결되었습/NNG', '다/EC', '‘/SS', '라고/JKQ', '나오/VV', '어/EC', '있/VX', '습니다/EC', '그럼/MAJ', '학사/NNG', '공지/NNG', '에/JKB', '나오/VV', '어/EC', '있/VX', '는/ETM', '대로/NNB', '재수강/NNP', '할/NNB', '이전/NNG', '성적/NNG', '을/JKO', '선택/NNG', '하/XSV', '어/EC', '재수강/NNP', '연결/NNG', '을/JKO', '하/VV', 'ᆯ/ETM', '필요/NNG', '없이/MAG', '아무것/NNG', '도/JX', '안/MAG', '하/VV', '어도/EC', '되/VV', 'ᆫ다는/ETM', '거/NNB', '이/VC

In [20]:
print(norm_content[31])
print(kiwi.analyze(norm_content[31],1))

청강은 학점으로 안들어가는 거 맞나요 그럼 만약 현재 최대 학점임에도 불구하고 청강 할 수 있는 거죠
[([], 0.0)]


In [13]:
## 불용어 처리 ##

file = open(r'E:\Programming\python\NLP\howls_nlp\Chatbot\자료\stop_words.txt','r', encoding = 'utf8')
lines = file.readlines()
file.close()
stop_words = lines[0].split()

#texts = [[word for word in document if word not in stop_words]
#            for document in lem_cont]

#print(texts[:10])

listed_cont = [[comb.split('/') for comb in sent] for sent in lem_cont]

main_texts = []
non_mean = []
for sent in listed_cont:
    texts = []
    for word in sent:
        # 한 글자 제거
        if len(word[0]) > 1:
            # 조사 등 불필요한 거 제거
            if 'N' in word[-1]:
                texts.append(word[0])
            elif '@' in word[-1]:
                texts.append(word[0])
            elif 'V' in word[-1]:
                texts.append(word[0])
            else:
                non_mean.append(word[0])
        else:
            non_mean.append(word[0])
    
    main_texts.append(texts)

lemmatized_question = []

for noun in main_texts:
    txt = ' '.join(noun)
    lemmatized_question.append(txt)

print(lemmatized_question[:10])

merged_df['LEMMA'] = lemmatized_question

merged_df

['이중전공 부전공 차이 대하 이중전공 학위 졸업장 표시 부전공 졸업장 표시', '재수강 연결 종합 정보 시스템 수업 수강 관리 수강 과목 선택 메뉴 들어가 안내 사항 이전 성적 연결 현황 동일 교과 자동연결되었습 나오 학사 공지 나오 대로 재수강 이전 성적 선택 재수강 연결 필요 아무것 감사', '수고 다르 아니 제가 평점 학기 수업 평점 올리 졸업 학점 134 학점 초과 문제', '처음 이중전공 신청 이중전공 경영 변경 시도 자체 시험 떨어지 변경 실패 지나 겨울 방학 2019 부전공 경영 선택 2020년 학기 이중전공 변경 기간 경영 신청 한번 종합 정보 시스템 들어가 사진 내요 신경 변경 기간 경영 신청', '이중전공 바꾸 바꾸 학점 소멸', '등록 휴학 자퇴 경우 이번 학기 등록금 환불 가능', '졸업 유예 기간 동안 학점 취득 가능', '인간 사회 학점 짜리 과목 씨쁠 이거 언어 문학 학점 짜리 페논 과목 재수강 연결 가능', '2019 학기 학기 마치 사정 학점 상태 휴학 전공 이중전공 학점 채운 상황 취득 학점 132 학점 학점 모자라 추가 학기 상황 경우 2020 여름 계절 학기 학점 수강 수료 가능 답변 부탁 드리', '교양 영어 수강 학생 온라인 과제 코드']


,QUESTION,ANSWER,TAGSET,LEMMA
0,이중전공이랑 부전공 차이에 대해서 궁금해요이중전공은 학위졸업장 표시인건 알겠는데부전...,NaN,NaN,이중전공 부전공 차이 대하 이중전공 학위 졸업장 표시 부전공 졸업장 표시
1,재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴를 들어갔는데안내사...,NaN,NaN,재수강 연결 종합 정보 시스템 수업 수강 관리 수강 과목 선택 메뉴 들어가 안내 사...
2,항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라 막학기에 수업을 더 ...,NaN,NaN,수고 다르 아니 제가 평점 학기 수업 평점 올리 졸업 학점 134 학점 초과 문제
3,제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 변경하려고 시도했는...,NaN,NaN,처음 이중전공 신청 이중전공 경영 변경 시도 자체 시험 떨어지 변경 실패 지나 겨울...
4,이중전공을 바꾸고 싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요 아...,NaN,NaN,이중전공 바꾸 바꾸 학점 소멸
...,...,...,...,...
6885,중외통 17학번인데요이중전공 어떤 걸로 하면 좋은지 추천해주세요,['아 그렇군요! 감사합니다!!'],NaN,중외 17 학번 이중전공 추천
6886,공대생 이중전공으로 언어할까 하는데 힘듦 언어추천좀,['스페인어 했다가 힘들어서 영어로 바꾼 사람은 봤음'],NaN,공대 이중전공 언어 힘들 언어 추천
6887,국제통상학과 이중전공생인데 걱정이 많이 된다,"['저도요.... 진짜 국통법 때문에 미칠거 같네요 ,', '나두...', '국통 ...",NaN,국제 통상 학과 이중전공 걱정
6888,경영 후기 이중전공 신청할 때이중전공 떨어져서 부전공 경영인데후기 이중전공 경영 신...,"['부전공학점 다 채운다음 이중전공이랑 차이나는 전공점수 채워야되요', '한마디로 ...",NaN,경영 후기 이중전공 신청 이중전공 떨어지 부전공 경영 후기 이중전공 경영 신청 경영...


In [14]:
##Tagset 달기
print(main_texts[:100])
tagset = []
for sent in main_texts:
    tags = []
    for word in sent:
        if '이중전공' == word:
            tags.append("DM")
        elif '부전공' == word:
            tags.append("SM")
        elif '전공심화' == word:
            tags.append("MD")
        elif '융합전공' == word:
            tags.append("UM")
        else:
            pass
    if not tags:
        tags.append("UK")
    tagset.append(tags)

tagsets = []
for sent_tag in tagset:
    tagsets.append(list(set(sent_tag)))

for tags in tagsets:
    if len(tags) > 1:
        del tags[-1]
print(tagsets[:10])


merged_df['TAGSET'] = tagsets

merged_df

[['이중전공', '부전공', '차이', '대하', '이중전공', '학위', '졸업장', '표시', '부전공', '졸업장', '표시'], ['재수강', '연결', '종합', '정보', '시스템', '수업', '수강', '관리', '수강', '과목', '선택', '메뉴', '들어가', '안내', '사항', '이전', '성적', '연결', '현황', '동일', '교과', '자동연결되었습', '나오', '학사', '공지', '나오', '대로', '재수강', '이전', '성적', '선택', '재수강', '연결', '필요', '아무것', '감사'], ['수고', '다르', '아니', '제가', '평점', '학기', '수업', '평점', '올리', '졸업', '학점', '134', '학점', '초과', '문제'], ['처음', '이중전공', '신청', '이중전공', '경영', '변경', '시도', '자체', '시험', '떨어지', '변경', '실패', '지나', '겨울', '방학', '2019', '부전공', '경영', '선택', '2020년', '학기', '이중전공', '변경', '기간', '경영', '신청', '한번', '종합', '정보', '시스템', '들어가', '사진', '내요', '신경', '변경', '기간', '경영', '신청'], ['이중전공', '바꾸', '바꾸', '학점', '소멸'], ['등록', '휴학', '자퇴', '경우', '이번', '학기', '등록금', '환불', '가능'], ['졸업', '유예', '기간', '동안', '학점', '취득', '가능'], ['인간', '사회', '학점', '짜리', '과목', '씨쁠', '이거', '언어', '문학', '학점', '짜리', '페논', '과목', '재수강', '연결', '가능'], ['2019', '학기', '학기', '마치', '사정', '학점', '상태', '휴학', '전공', '이중전공', '학점', '채운', '상황', '취득', '학점', '132', '학점', '학점', '모자라', '

,QUESTION,ANSWER,TAGSET,LEMMA
0,이중전공이랑 부전공 차이에 대해서 궁금해요이중전공은 학위졸업장 표시인건 알겠는데부전...,NaN,"[DM, UK]",이중전공 부전공 차이 대하 이중전공 학위 졸업장 표시 부전공 졸업장 표시
1,재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴를 들어갔는데안내사...,NaN,[UK],재수강 연결 종합 정보 시스템 수업 수강 관리 수강 과목 선택 메뉴 들어가 안내 사...
2,항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라 막학기에 수업을 더 ...,NaN,[UK],수고 다르 아니 제가 평점 학기 수업 평점 올리 졸업 학점 134 학점 초과 문제
3,제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 변경하려고 시도했는...,NaN,"[DM, UK]",처음 이중전공 신청 이중전공 경영 변경 시도 자체 시험 떨어지 변경 실패 지나 겨울...
4,이중전공을 바꾸고 싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요 아...,NaN,[DM],이중전공 바꾸 바꾸 학점 소멸
...,...,...,...,...
6885,중외통 17학번인데요이중전공 어떤 걸로 하면 좋은지 추천해주세요,['아 그렇군요! 감사합니다!!'],[DM],중외 17 학번 이중전공 추천
6886,공대생 이중전공으로 언어할까 하는데 힘듦 언어추천좀,['스페인어 했다가 힘들어서 영어로 바꾼 사람은 봤음'],[DM],공대 이중전공 언어 힘들 언어 추천
6887,국제통상학과 이중전공생인데 걱정이 많이 된다,"['저도요.... 진짜 국통법 때문에 미칠거 같네요 ,', '나두...', '국통 ...",[DM],국제 통상 학과 이중전공 걱정
6888,경영 후기 이중전공 신청할 때이중전공 떨어져서 부전공 경영인데후기 이중전공 경영 신...,"['부전공학점 다 채운다음 이중전공이랑 차이나는 전공점수 채워야되요', '한마디로 ...","[DM, UK]",경영 후기 이중전공 신청 이중전공 떨어지 부전공 경영 후기 이중전공 경영 신청 경영...


In [71]:
# csv로 출력


merged_df.to_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\results\preprocessed_question.csv", index=False, encoding="utf-8")